# Regression Coding Challenge 2: Pre-processing the Raw Titanic Dataset
© Explore Data Science Academy


Within the previous challenge, we began our journey towards creating a regression model capable of predicting the survivors of the historic sinking of the Titanic. 

For today's challenge, we expand upon this exploration by cleaning our data and performing feature engineering; ensuring that it is ready for use in various modelling strategies.  

<br></br>

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://i.kinja-img.com/gawker-media/image/upload/fhygjbyvm8rorm1uyg6s.jpg"
     alt="Barnicles on your ship :( "
     style="float: center; padding-bottom=0.5em"
     width=600px/>
Time to clean some ship data!
</div>



## Honour Code

I **YOUR NAME**, **YOUR SURNAME**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.




## Pre-requisites 

Before getting your hands dirty in this challenge (or erm... cleaning them), please ensure that the following steps have been taken:

 1. You have completed _Regression Coding Challenge 1_. The coding challenges are designed to build upon one another, mimicking a process you will follow for most of your data science projects. 
 
 2. You have reviewed and are comfortable with the Titanic dataset contents. A further description of each field within the dataset can be found [here](https://www.kaggle.com/c/titanic/data)
 
With these steps out of the way, we're ready to begin!

### Imports 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Data Review

In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/regression_sprint/titanic_train_raw.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/regression_sprint/titanic_test_raw.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
test_df.head()

## Question 1: Handling Missing Data

Having a comprehensive view of our dataset afforded by its exploration, we are ready to begin the first step of cleaning our data by considering cases of missing values.

### Question 1.1: Removing Features

In Question 1 of Challenge 1, we learnt that certain columns within our dataset had a high percentage of missing values. While there are numerous ways of handling such missing data, sometimes it is appropriate to discard them altogether. 

With this in mind, write a function called `drop_columns` which removes a column from a pandas dataframe if the percentage of missing values is greater than a given `threshold` value. Additionally, a user should also be able to provide a `unique_value_threshold` which removes a column if the percentage of unique values in that column is below the `unique_value_threshold`.    

***Function arguments:***
 - `input_df` -> input `Pandas` DataFrame.
 - `threshold`-> python `float`, `threshhold` $\in [0, 100.0]$.
 - `unique_value_threshold` -> python `float`, `unique_value_threshold` $\in [0, 100.0]$.

***Function Specifications:***
 - Name the function `drop_columns`
 - Must take any Pandas `DataFrame` as input and return a `DataFrame` as output.
 - Must remove one or more columns which exceed the drop `threshold`, as well as any columns whose percentage of unique values is below the `unique_value_threshold`. 
 - If no columns exceed the `threshold` or have a unique value percentage less than `unique_value_threshold`, a dataframe identical to the input must be returned.  
 - **NaN values should be excluded when computing the percentage of unique values**

In [ ]:
### START FUNCTION
def drop_columns(input_df, threshold, unique_value_threshold):
    # your code here
    output_df = input_df
    holder = list(input_df.columns)
    holder2 = []
    for x in holder:
        if ((input_df[x].isna().sum()/len(input_df[x]))*100 > threshold):
            holder2.append(x)
        elif ((input_df[x].nunique()/len(input_df[x]))*100< unique_value_threshold):
            holder2.append(x)
    output_df = output_df.drop(holder2, axis=1)
    
    return output_df
        

### END FUNCTION

In [ ]:
drop_columns(train_df, 60, 0.5).columns == ['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare']

_**Expected Outputs:**_
```python
drop_columns(train_df, 60, 0.5).columns == ['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare']

drop_columns(train_df, 80, 15).columns == ['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin']
```

### Question 1.2: Data Imputation 

In Question 2 of Challenge 1, we saw that it is possible to replace or [impute](https://en.wikipedia.org/wiki/Imputation_(statistics)) missing data based on simple population statistics such as the mean, median or mode of the targeted feature.

When a friend of yours hears that you are imputing the `'Age'` column by simply taking the *mean* or *median* of the non-missing data, she decides to show-off the following code which produces two telling plots:    

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1, 2, 1)
sns.boxenplot(train_df['Pclass'], train_df['Age'], hue=train_df['Survived'])
plt.subplot(1, 2, 2)
sns.boxenplot(train_df['Sex'], train_df['Age'], hue=train_df['Survived'])
plt.show()

In [ ]:
grid = sns.FacetGrid(train_df, row='Pclass', col='Sex', height=2.0, aspect=3.0)
grid.map(plt.hist, 'Age', bins=30)
grid.add_legend()
plt.show()

Using these plots, your friend argues that there are correlations between the `'Pclass'`, `'Sex'` and `'Age'` features within the dataset, and that `'Age'` shows influence in predicting a given passenger's survival. 

Aware of this new knowledge, you decide to create a function, `conditional_impute`, which performs imputation upon an `age` column within a Pandas `DataFrame` based upon both `Sex` and `Pclass` columns. Given that you are uncertain which statistic best models your data, you decide to once again to add a `choice` parameter which allows either the *mean* or *median* to be used for imputing the data.   

***Function arguments:***
 - `input_df` -> input `Pandas` DataFrame. 
 - `choice`-> Python `string` of either `'mean'` or `'median'`. Default is `'median'`. 

***Function Specifications:***
 - Name the function `conditional_impute`
 - Must take a Pandas `DataFrame` as input and return a `DataFrame` as output.
 - Should impute `Age` values based upon the descriptive statistic method specified by `choice`.
 - Round all imputed answers to 1 decimal point. 
 - If `choice` is not either `'mean'` or `'median'`,  a `ValueError` must be raised.  

In [ ]:
### START FUNCTION
def conditional_impute(input_df, choice='median'):
    # your code here
    output_df = input_df
    missing = output_df[output_df['Age'].isnull()]
    meaner = output_df.groupby(['Sex','Pclass'])['Age'].mean()
    medianer = output_df.groupby(['Sex','Pclass'])['Age'].median()
    def fill_missing_mean(row):
        if pd.isnull(row['Age']):
            return meaner[row['Sex'],row['Pclass']]
        else:
            return row['Age']
    def fill_missing_median(row):
        if pd.isnull(row['Age']):
            return medianer[row['Sex'],row['Pclass']]
        else:
            return row['Age']
    
    if choice == 'mean':
        output_df['Age'] =output_df.apply(fill_missing_mean, axis=1)
    elif choice == 'median':
        output_df['Age'] =output_df.apply(fill_missing_median, axis=1)
    else:
        raise ValueError()
    return output_df

### END FUNCTION

In [ ]:
conditional_impute(train_df, choice='mean')[['Name','Age']].tail()

_**Expected Outputs:**_
```python
conditional_impute(train_df, choice='median')[['Name','Age']].tail()

```

><table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Name</th>      <th>Age</th>    </tr>  </thead>  <tbody>    <tr>      <th>886</th>      <td>Montvila, Rev. Juozas</td>      <td>27.0</td>    </tr>    <tr>      <th>887</th>      <td>Graham, Miss. Margaret Edith</td>      <td>19.0</td>    </tr>    <tr>      <th>888</th>      <td>Johnston, Miss. Catherine Helen "Carrie"</td>      <td>21.5</td>    </tr>    <tr>      <th>889</th>      <td>Behr, Mr. Karl Howell</td>      <td>26.0</td>    </tr>    <tr>      <th>890</th>      <td>Dooley, Mr. Patrick</td>      <td>32.0</td>    </tr>  </tbody></table>

<br><br>

```python
conditional_impute(train_df, choice='mean')[['Name','Age']].tail()
```

> <table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Name</th>      <th>Age</th>    </tr>  </thead>  <tbody>    <tr>      <th>886</th>      <td>Montvila, Rev. Juozas</td>      <td>27.0</td>    </tr>    <tr>      <th>887</th>      <td>Graham, Miss. Margaret Edith</td>      <td>19.0</td>    </tr>    <tr>      <th>888</th>      <td>Johnston, Miss. Catherine Helen "Carrie"</td>      <td>21.8</td>    </tr>    <tr>      <th>889</th>      <td>Behr, Mr. Karl Howell</td>      <td>26.0</td>    </tr>    <tr>      <th>890</th>      <td>Dooley, Mr. Patrick</td>      <td>32.0</td>    </tr>  </tbody></table>

## Question 2: Feature Engineering 

Having considered missing data, we can now further prepare our dataset for use within a model by performing feature engineering.

### Question 2.1: What's in a name?

When you originally received the titanic dataset, you were excited to see the `Name` feature included, as you believed this might be another source of information to help infer a person's social status on the ship. To use this feature however, we need to extract an individual's _title_ from his/her name, as calling `train_df['Name'].unique().shape` tells us that every `Name` entry within our train dataset is currently unique. 

Go ahead and perform this transformation by writing a function called `extract_title`, which adds an extra `Title` column to our dataframe into which is placed a person's given title found within the `Name` column. 

Examples of title extraction; 
 - `Braund, Mr. Owen Harris` maps to a title of `Mr.`
 - `Heikkinen, Miss. Laina` maps to a title of `Miss.`

***Function arguments:***
 - `input_df` -> input `Pandas` DataFrame.

***Function specifications:***
 - Name the function `extract_title`
 - Must take any Pandas `DataFrame` as input and return a `DataFrame` as output with an additional `Title` column.
 - Assume that `input_df` represents a DataFrame possessing a `'Name'` column, with each corresponding row entry being a `string`-based name containing exactly one title. 
 - Assume that a title is represented by a word with two or more characters ending in a `.`

In [ ]:
### START FUNCTION
def extract_title(input_df):
    # your code here
    output_df = input_df.copy()
    
    output_df['Name_hold'] = output_df['Name'].str.split()
    index = output_df.index
    for x in index:
        title = output_df.at[x, 'Name_hold']
        for y in title:
            if y[-1] == '.' && len(y) >=3:
                output_df.at[x, 'Title'] = y
    return output_df
### END FUNCTION

In [ ]:
extract_title(train_df)['Title'].unique() == ['Mr.', 'Mrs.', 'Miss.', 'Master.',
                                              'Don.', 'Rev.', 'Dr.', 'Mme.','Ms.',
                                              'Major.', 'Lady.', 'Sir.', 'Mlle.',
                                              'Col.', 'Capt.','Countess.', 'Jonkheer.']

_**Expected Outputs:**_
```python

extract_title(train_df)['Title'].unique() == ['Mr.', 'Mrs.', 'Miss.', 'Master.',
                                              'Don.', 'Rev.', 'Dr.', 'Mme.','Ms.',
                                              'Major.', 'Lady.', 'Sir.', 'Mlle.',
                                              'Col.', 'Capt.','Countess.', 'Jonkheer.']


extract_title(test_df)['Title'].unique() == ['Mr.', 'Mrs.', 'Miss.', 'Master.', 
                                             'Ms.', 'Col.', 'Rev.', 'Dr.','Dona.']

``` 

### Question 2.2: Feature Selection

Having your `'Title'` column prepared, you perform a quick examination to see the distribution of its categories with relation to the gender of a passenger.

(***Note:*** Even if you weren't able to solve Q2.1, *we have imported the correct dataframe*, `title_df` in the code section beneath for your continued use.)

In [ ]:
# Load the correct form of the train.csv for Q 2.2
title_df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/regression_sprint/titanic_train_title.csv')
pd.crosstab(index=title_df['Title'], columns=title_df['Sex'])

From this observation, you realise that a just small group of categories represent a considerable majority of datapoints within the column. Using this outcome in your favour, you decide to only preserve the more common titles and to group the remaining items into a single category called `'Uncommon'`. 


To accomplish your objective, write a function called `group_titles` which takes in your dataframe containing the `Title` column, along with a selection of `uncommon_titles`, and returns the dataframe with the modified `Title` column in which the updated groupings are reflected.

***Function arguments:***
 - `input_df` -> input `Pandas` DataFrame.
 - `uncommon_titles` -> A 1-D `list` of Python `strings` representing the categories of titles to be grouped into the `"Uncommon"` category. 
***Function specifications:***
 - Name the function `group_titles`
 - Must take any Pandas `DataFrame` as input and return a `DataFrame` with the `Title` column containing newly regrouped categories.
 - Assume that `input_df` represents a DataFrame possessing a `'Title'` column, with each corresponding row entry being a `string`-based title. 

In [ ]:
### START FUNCTION
def group_titles(input_df, uncommon_titles):
    # your code here
    output_df = input_df
    for x in uncommon_titles:
        output_df['Title'] = output_df['Title'].replace(x, 'Uncommon')
    
    return output_df
    

### END FUNCTION

In [ ]:
uncommon_titles = ['Don.', 'Rev.', 'Mme.','Major.', 'Sir.', 'Mlle.','Countess.', 'Jonkheer.']
group_titles(title_df, uncommon_titles=['Don.', 'Rev.', 'Mme.','Major.', 'Sir.', 'Mlle.','Countess.', 'Jonkheer.'])['Title'].unique()

_**Expected Outputs:**_
```python
group_titles(title_df, uncommon_titles=['Don.', 'Rev.', 'Mme.','Major.', 'Sir.', 'Mlle.','Countess.', 'Jonkheer.'])['Title'].unique() == 
['Mr.', 'Mrs.', 'Miss.', 'Master.', 'Uncommon', 'Dr.', 'Ms.',
       'Lady.', 'Col.', 'Capt.']
```

### Question 2.3: Feature Encoding

Finally, you further decide that it's time to encode your new categorical features within the `'Title'` column using [dummy variables.](https://en.wikipedia.org/wiki/Dummy_variable_(statistics)) 

Write a function called `encode_titles` which encodes the various categories contained within the `Title` column of an input DataFrame as numerically-based dummy variables. 

***Function arguments:***
 - `input_df` -> input `Pandas` DataFrame.

***Function specifications:***
 - Name the function `encode_titles`
 - Must take any Pandas `DataFrame` as input and return a `DataFrame` as output with an expanded set of columns representing dummy variables corresponding to the newly regrouped `Title` categories.
 - Assume that `input_df` represents a DataFrame possessing a `'Title'` column, with each corresponding row entry being a `string`-based title. 
 - **NB!** Ensure that your dummy encoding avoids the [dummy variable trap](https://stattrek.com/multiple-regression/dummy-variables.aspx). As a ***hint***, look at the documentation for the Pandas [get_dummies()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) function.

(***Note:*** Even if you weren't able to solve Q2.2, *we have imported the correct dataframe*, `title_regrouped_df` in the code section beneath for your continued use.)

In [ ]:
# Load the correct form of the train.csv for Q 2.2
title_regrouped_df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/regression_sprint/titanic_train_title_regrouped.csv')

Function implementation:

In [ ]:
### START FUNCTION
def dummy_encode_titles(input_df):
    # your code here
    return pd.get_dummies(input_df, drop_first=True)

### END FUNCTION

In [ ]:
encode_df = dummy_encode_titles(title_regrouped_df)
dummy_cols = [col for col in encode_df if col.startswith('Title')]
encode_df[dummy_cols].head()

_**Expected Outputs:**_
```python
encode_df = dummy_encode_titles(title_regrouped_df)
dummy_cols = [col for col in encode_df if col.startswith('Title')]
encode_df[dummy_cols].head()
```

><table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Title_Miss.</th>      <th>Title_Mr.</th>      <th>Title_Mrs.</th>      <th>Title_Uncommon</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>0</td>      <td>1</td>      <td>0</td>      <td>0</td>    </tr>    <tr>      <th>1</th>      <td>0</td>      <td>0</td>      <td>1</td>      <td>0</td>    </tr>    <tr>      <th>2</th>      <td>1</td>      <td>0</td>      <td>0</td>      <td>0</td>    </tr>    <tr>      <th>3</th>      <td>0</td>      <td>0</td>      <td>1</td>      <td>0</td>    </tr>    <tr>      <th>4</th>      <td>0</td>      <td>1</td>      <td>0</td>      <td>0</td>    </tr>  </tbody></table>